In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq

model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C15C481090>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C15C481C60>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="this is archita")])


AIMessage(content='Hello Archita! 👋\n\nIs there anything I can help you with today? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34, 'completion_time': 0.038181818, 'prompt_time': 9.5449e-05, 'queue_time': 0.023037970999999997, 'total_time': 0.038277267}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bb2e5082-ca5d-425e-a84b-047a2f0d6035-0', usage_metadata={'input_tokens': 13, 'output_tokens': 21, 'total_tokens': 34})

In [6]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="this is archita"),
    AIMessage(content="nice to meet you archita! 👋\n\nHow can I help you today? 😊\n'"),
    HumanMessage(content="what is my name ?")])

#it is able to remember the previous context

AIMessage(content='Your name is Archita.  😄  \n\nI remember that from our introduction! \n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 47, 'total_tokens': 68, 'completion_time': 0.038181818, 'prompt_time': 0.000525699, 'queue_time': 0.020591969999999998, 'total_time': 0.038707517}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-03293240-22c9-4889-a92a-b7d29eb4701f-0', usage_metadata={'input_tokens': 47, 'output_tokens': 21, 'total_tokens': 68})

In [7]:
llm=model.invoke([HumanMessage(content="this is archita")])
response=llm[0].content

TypeError: 'AIMessage' object is not subscriptable

In [8]:
type(llm)
#how to just type the llm content message

langchain_core.messages.ai.AIMessage

In [9]:
#session
#message history
!pip install langchain_community

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

#to retreive all our chat history content
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

#to interact with our llm model based on this chat history

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [11]:
config={"configurable":{"session_id":"chat1"}}

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is siri. I am student")],
    config=config
)

In [13]:
response.content

"Hello Siri! It's nice to meet you.  \n\nBeing a student is a lot of work, but also very rewarding. What are you studying?  What are some of your favorite things about being a student?\n"

In [14]:
response1=with_message_history.invoke(
    [HumanMessage(content="whats my name ?")],
    config=config
)
response1.content

"Your name is Siri. You told me at the beginning of our conversation! 😊  \n\nIs there anything else I can help you with?  Perhaps you'd like to talk about what you're studying?\n"

In [15]:
#change the session id

config1={"configurable":{"session_id":"chat1"}}

response2=with_message_history.invoke(
    [HumanMessage(content="whats my name ?")],
    config=config1
)
response2.content


"Your name is Siri.  \n\nI remember!  Is there something else you'd like to chat about?  Perhaps you have a question about your studies, or maybe you want to tell me about your hobbies? 📚🎨🎮  I'm here to listen! 😊  \n\n\n\n\n"

In [16]:
config1={"configurable":{"session_id":"chat2"}}

response2=with_message_history.invoke(
    [HumanMessage(content="whats my name ?")],
    config=config1
)
response2.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know! 😊\n"

In [17]:
config2={"configurable":{"session_id":"chat2"}}

response2=with_message_history.invoke(
    [HumanMessage(content="my name is john ?")],
    config=config2
)
response2.content

"Hi John! 👋 \n\nIt's nice to meet you.  Is there anything I can help you with today? 😊  \n\n"

In [18]:
#session id concept is very very important....... read a lot about it..

In [19]:
#promppt templates

#it is used to turn raw user information that the LLM can work with. 

In [20]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt= ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant"),
        MessagesPlaceholder(variable_name="messages")

    ]
)

chain=prompt|model

#messsages= [HumanMessage(content="hi my name is siri")]


chain.invoke({"messages":[HumanMessage(content="hi my name is siri")]})

AIMessage(content="Hello Siri! 👋 \n\nIt's nice to meet you. What can I do to help you today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 19, 'total_tokens': 48, 'completion_time': 0.052727273, 'prompt_time': 7.866e-05, 'queue_time': 0.02467206, 'total_time': 0.052805933}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8bda910b-1bae-4550-bde0-fdb225ccb467-0', usage_metadata={'input_tokens': 19, 'output_tokens': 29, 'total_tokens': 48})

In [21]:
with_message_history= RunnableWithMessageHistory(chain,get_session_history)

config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
        [HumanMessage(content="hi my name is siri")],
        config=config

    )

In [22]:
response

AIMessage(content="Hi Siri! It's nice to meet you.  \n\nWhat can I do to help you today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 19, 'total_tokens': 47, 'completion_time': 0.050909091, 'prompt_time': 8.385e-05, 'queue_time': 0.024271219, 'total_time': 0.050992941}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1d3e6e2a-1c10-42dc-a7c5-77aa3192fa82-0', usage_metadata={'input_tokens': 19, 'output_tokens': 28, 'total_tokens': 47})

In [29]:
#adding complexity

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","you are a scientist answer to your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [33]:
response=chain.invoke({"messages":[HumanMessage(content="hi my name is sumanth")],"language":"telugu"})
response.content

'నేను ఒక శాస్త్రవేత్తగా మీ సమాధానాలను అందించడానికి మీ కోసం సిద్ధంగా ఉన్నాను, సుమంత్. 😊 \n\nమీరు నన్ను ఏ విషయంలోనైనా అడిగితే నా జ్ఞానాన్ని ఉపయోగించి ప్రయత్నిస్తాను. \n\nమీరు ఏ విషయం గురించి తెలుసుకోవాలనుకుంటున్నారు? 🤔 \n\n'

In [34]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"

)

In [35]:
config=config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke({"messages":[HumanMessage(content="hi my name is sumanth")],
                                      "language":"telugu"},config=config)
response.content


"హాయ్ సుమంత్, నేను ఒక వైద్యుడిని నేను మీ ప్రశ్నలకు నా సరికొత్త జ్ఞానం ప్రకారం సమాధానం ఇచ్చేందుకు ప్రయత్నిస్తాను. మీరు ఏ విషయం గురించి తెలుసుకోవాలనుకుంటున్నారు? \n\n(Hi Sumanth, I am a scientist. I will try my best to answer your questions based on my knowledge. What would you like to know?) \n\n\nLet me know your question and I'll do my best to help! 😊\n\n"

In [36]:
response=with_message_history.invoke({"messages":[HumanMessage(content="what is my name ?")],
                                      "language":"telugu"},config=config)
response.content


'మీ పేరు సుమంత్ అని చెప్పారు. 😊 \n\n(You said your name is Sumanth.) \n'

In [40]:
from langchain_core.messages import SystemMessage,HumanMessage,trim_messages

trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [41]:
messages=[
    SystemMessage(content="you are a baby"),
    HumanMessage(content="hi this is sumanth"),
    AIMessage(content="hi how can i help you"),
    HumanMessage(content="I like ice cream"),
    AIMessage(content="so what !!"),
    HumanMessage(content="why are you like this talk properly"),
    AIMessage(content="i am the way i am"),
    HumanMessage(content="ok your wish"),
    AIMessage(content="ok i also like ice cream"),
    HumanMessage(content="which flavour do you like the most"),
    AIMessage(content="I like the flavour you dislike the most"),

]

In [42]:
trimmer.invoke(messages)

c:\Users\siri_\mylearning\langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\siri_\mylearning\langchain\venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\siri_\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mo

[SystemMessage(content='you are a baby', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='ok your wish', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ok i also like ice cream', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='which flavour do you like the most', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I like the flavour you dislike the most', additional_kwargs={}, response_metadata={})]

In [50]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

from langchain_groq import ChatGroq

model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

response=chain.invoke({
    "messages":messages +[HumanMessage(content="what ice cream do i like")],
    "language":"english"
})

In [51]:
response.content

"As a scientist, I don't have personal preferences or access to information about you, including your favorite ice cream flavor. \n\nCould you tell me what flavors you enjoy?  🍦😊  \n\n"

In [52]:
response=chain.invoke({
    "messages":messages +[HumanMessage(content="do i like ice cream")],
    "language":"english"
})

In [53]:
response.content

"Hmmm... that's a tricky one!  As a baby scientist, I can't know what *you* like.  \n\nMaybe you like chocolate ice cream? Or strawberry?  Or maybe something super fancy like pistachio?\n\nTell me, what's your favourite flavour?  🍦  \n"

In [54]:
#wrapping into message history

with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config=config={"configurable":{"session_id":"chat5"}}
response=with_message_history.invoke({"messages":[HumanMessage(content="hi my name is sumanth")],
                                      "language":"telugu"},config=config)
response.content

"నేను ఒక సైంటిస్టుగా, మీకు సహాయం చేయడానికి ఇక్కడ ఉన్నాను, సుమంత్. \n\nమీరు ఏ విషయంలో సహాయం అవసరం అని చెప్పండి. నేను మీ ప్రశ్నలకు సమాధానం ఇవ్వడానికి, విజ్ఞానం साझा करने కి, లేక ప్రయోగాలు సూచించడానికి ప్రయత్నిస్తాను. \n\n\n(I am a scientist, and I'm here to help you, Sumanth. \n\nTell me what you need help with. I will try my best to answer your questions, share knowledge, or suggest experiments.)\n\n"